# 2週目の内容 (画像処理、データ処理)
画像処理の時に役に立ったコレ　https://note.com/kiyo_ai_note/n/ndb9df1fa3d24

## 画像データの準備

week2.ipynbと同じ階層にdata/を配置。train, valと分けて、わんちゃんとねこちゃんのデータをそれぞれ80:20で分割して挿入した。いぬねこは[Kaggle](https://www.kaggle.com/datasets/tongpython/cat-and-dog?resource=download)からもらった。

``` zsh
data
├── train
│   ├── cat
│   └── dog
└── val
    ├── cat
    └── dog
```

## Datasetクラス
torchvision.datasets.ImageFolder()：torchvisionのクラスで、画像をフォルダ構造に基づいて読み込むために使用されます。このクラスは、フォルダ名をクラスラベルとして画像データを自動的にラベリングし、データのロードを効率的に行えるようにします。

引数
- root: データセットのルートディレクトリ
- transform: 画像を適用する変換処理 (リサイズとか正規化)
- loader: 画像を読み込むための関数(デフォルトはPIL)

In [3]:
import torch
import torchvision

train_image_dir = "data/train"
val_image_dir = "data/val"

train_dataset = torchvision.datasets.ImageFolder(root=train_image_dir)
val_dataset = torchvision.datasets.ImageFolder(root=val_image_dir)

print(train_dataset)
print(val_dataset)

ModuleNotFoundError: No module named '_lzma'

In [ ]:
img1, label1 = train_dataset[0]
img2, label2 = train_dataset[-1]

print("img1:{}".format(img1))
print("label1:{}".format(label1))
print("img2:{}".format(img2))
print("label2:{}".format(label2))

In [ ]:
import matplotlib.pyplot as plt

plt.show(img1)

## Datasetの変換
### 画像サイズを変更
transforms.RandomResizedCrop()を使うよ
引数は下の順に入れていく
- size (int or sequence) ：トリミング後の出力画像サイズ（値がint型の場合は(size, size)の寸法となる）
- scale (tuple of python:float)：リサイズする上限・下限を設定する。
- ratio (tuple of python:float)：アスペクト比(ランダム)の上限・下限を設定
- interpolation (InterpolationMode)：ー

In [ ]:
from torchvision import transforms

resize = transforms.RandomResizedCrop(32)
img1_re = resize(img1)
print("size:{}".format(img1_re.size))
plt.show(img1_re)

### 画像をPyTorchテンソルへ変換する
transformers.ToTensorでできるよ

In [ ]:
to_tensor = transforms.ToTensor()
img1_t = to_tensor(img1_re)
print(img1_t.shape)

### サイズを変更し、PyTorchテンソルに変換したデータセットを作成する

transforms.Compose() : 

torchvision.datasets.ImageFolder() : 

.format() : 


In [4]:
import torch
import torchvision
from torchvision import transforms

image_size = 32
train_image_dir = "data/train"
val_image_dir = "data/val"

data_transform = {
    'train' : transforms.Compose([transforms.RandomResizedCrop(image_size),
                                  transforms.ToTensor()]),
    'val' : transforms.Compose([transforms.RandomResizedCrop(image_size),
                                transforms.ToTensor()])
}

tensor_train_dataset = torchvision.datasets.ImageFolder(
                            root=train_image_dir,
                            transform=data_transform("train"))
tensor_val_dataset = torchvision.datasets.ImageFolder(
                        root=val_image_dir,
                        transform=data_transform["val"])

img1_t, label = tensor_train_dataset[0]
print("type:{}, label:{}, shape:{}, dtype:{}".format(
                                                type(img1_t),
                                                label,
                                                img1_t.shape,
                                                img1_t.dtype))

img2_t, label = tensor_train_dataset[0]
print("type:{}, label:{}, shape:{}, dtype:{}".format(
                                                type(img2_t),
                                                label,
                                                img2_t.shape,
                                                img2_t.dtype))



ModuleNotFoundError: No module named '_lzma'

In [ ]:
print("min:{}, max:{}".format(img1_t.min(), img1_t.max()))

In [ ]:
plt.imshow(img1_t.permute(1, 2, 0))

### データの正規化

正規化する理由は0±1あたりで線形である活性化関数を選択する際に使いやすいから！

In [ ]:
imgs = torch.stack([img_t for img_t, _ in tensor_train_dataset], dim=3)
print(imgs.shape)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)
print("mean:{}, std:{}".format(mean, std))

In [ ]:
import torch
import torchvision
from torchvision import transforms

image_size = 32
mean = (hage, hige, huge)
std = (foo, bar, hoo)
train_image_dir = "data/train"
val_image_dir = "data/val"

data_transform = {
    'train': transforms.Compose([transforms.RandomResizedCrop(image_size),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean, std)]),

    'val': transforms.Compose([transforms.RandomResizedCrop(image_size),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean, std)])
}

train_dataset = torchvision.datasets.ImageFolder(
                                        root=train_image_dir,
                                        transform=data_transform["train"])
val_dataset = torchvision.datasets.ImageFolder(
                                    root=train_image_dir,
                                    transform=data_transform["val"])


ModuleNotFoundError: No module named '_lzma'

In [ ]:
img, label = train_dataset[0]
plt.imshow(img.permute(1, 2, 0))

## 分類画像モデルの構築

In [10]:
import torch
import torch.nn as nn

device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
print("device:{}".format(device))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)

model = nn.Sequential(
    nn.Linear(3072, 512),
    nn.Tanh(),
    nn.Linear(512, 2),
    nn.LogSoftmax(dim=1)).to(device=device)

learning_rate = 1e-2
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.NLLLoss()
n_ephocs = 50

for epoch in range(n_ephocs):
    for imgs, labels in train_loader:
        imgs = imgs.to(device=device)
        labels = labels.to(device=device)
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        loss = torch.tensor(loss_fn(outputs, labels))
        # ほんとは loss = loss_fn(outputs, labels)だけど、backward()がtensorクラスのものだから認識しない。上のでいけるか？

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print("Epoch:{}, Loss:{:.3f}".format(epoch, float(loss)))

device:cpu


NameError: name 'train_dataset' is not defined

In [9]:
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=8, shuffle=False)
correct = 0
total = 0
with torch.no_grad():
    for imgs, label in val_loader:
        imgs = imgs.to(device=device)
        labels = labels.to(device=device)
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        correct += int((predicted == labels).sum())
    
print("Accuracy:{:.3f}".format(float(correct/total)))

SyntaxError: incomplete input (4032426503.py, line 5)

## 演習
### 1. エポック数 (学習回数)を変化させた時の損失関数の値と予測精度を調査する。

pytable使って表を作れそう
https://qiita.com/arakiii/items/5bece948dd459b53d0b8

In [ ]:
import torch

# requires_grad=True のテンソル
x = torch.tensor(2.0, requires_grad=True)

# y = x^2 の計算 (計算グラフが構築される)
y = x ** 2

# backward() の呼び出し
y.backward()

# 勾配を確認
print(x.grad)  # dy/dx = 2x = 4


### 2. 活性化関数を変えた場合の損失関数の値と予測制度を調査する。
ReLUにした場合　(epoch 10~100で10刻み)

### 3. ネットワークの構造を変えた場合の損失関数の値と予測精度を調査する。
活性化関数はReLUで